In [3]:
import zipfile
import json
import os
from collections import defaultdict
from tqdm import tqdm
import shutil
import random
import requests

In [5]:
data_folder = "/home/juan/CEIA/CEIA-ViT/TrabajosPracticos/TP_Final/data/raw/"

def get_data():
    os.makedirs(data_folder, exist_ok=True)

    urls = {
        "cocotext.v2.zip": "https://bgshih.github.io/cocotext/cocotext.v2.zip",
        "train2014.zip": "http://images.cocodataset.org/zips/train2014.zip"
    }

    for name_file, url in urls.items():
        ruta_destino = os.path.join(data_folder, name_file)
        if os.path.exists(ruta_destino):
            print(f"{name_file} ya existe. Omitiendo descarga.")
            continue

        print(f"Descargando {name_file}...")
        respuesta = requests.get(url, stream=True)
        total = int(respuesta.headers.get('content-length', 0))

        with open(ruta_destino, 'wb') as archivo, tqdm(
            desc=name_file,
            total=total,
            unit='B',
            unit_scale=True,
            unit_divisor=1024
        ) as barra:
            for datos in respuesta.iter_content(chunk_size=1024):
                archivo.write(datos)
                barra.update(len(datos))

    print("Descarga finalizada.")

get_data()


cocotext.v2.zip ya existe. Omitiendo descarga.
train2014.zip ya existe. Omitiendo descarga.
Descarga finalizada.


In [6]:
def count_images(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        return len([f for f in zipf.namelist() if f.lower().endswith('.jpg')])

def extract_anns(zip_path, json_filename='cocotext.v2.json'):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        with zipf.open(json_filename) as json_file:
            return json.load(json_file)
        
def contar_labeled(coco_text_data):
    etiquetadas = set()
    for ann_id, ann in coco_text_data['anns'].items():
        img_id = ann['image_id']
        if ann['legibility'] in ['legible', 'illegible']:
            etiquetadas.add(img_id)
    return len(etiquetadas)


In [8]:
train_zip = data_folder + 'train2014.zip'
annotations_zip = data_folder+'cocotext.v2.zip'

num_train_imgs = count_images(train_zip)

coco_text = extract_anns(annotations_zip)

num_etiquetadas = contar_labeled(coco_text)

print(f'Imágenes en train2014.zip: {num_train_imgs}')
print(f'Imágenes etiquetadas (con texto): {num_etiquetadas}')

Imágenes en train2014.zip: 82783
Imágenes etiquetadas (con texto): 23485


In [9]:
def image_id_to_train_filename(image_id):
    return f'COCO_train2014_{image_id:012d}.jpg'


def load_annotations(zip_path, json_filename='cocotext.v2.json'):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        with zipf.open(json_filename) as file:
            return json.load(file)


def get_annotated_image_ids(coco_data):
    annotated_ids = set()
    for ann in coco_data['anns'].values():
        annotated_ids.add(ann['image_id'])
    return annotated_ids


def extract_annotated_images(zip_path, output_dir, target_filenames):
    os.makedirs(output_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        zip_files = zipf.namelist()
        # Match full paths like 'train2014/COCO_train2014_000000001234.jpg'
        matched_files = [f for f in zip_files if os.path.basename(f) in target_filenames]

        if not matched_files:
            print(f'No matching files found in {zip_path}')
            return

        for file in tqdm(matched_files, desc=f'Extracting from {os.path.basename(zip_path)}'):
            zipf.extract(file, output_dir)

In [10]:
output_dir = '../data/raw/subset/train2014'

# load annotations and get annotated image IDs
coco_data = load_annotations(annotations_zip)
annotated_ids = get_annotated_image_ids(coco_data)

# convert IDs to expected filenames
train_filenames = set(image_id_to_train_filename(img_id) for img_id in annotated_ids)

# extract only annotated images from train2014.zip
extract_annotated_images(train_zip, output_dir, train_filenames)

Extracting from train2014.zip: 100%|██████████| 23485/23485 [00:11<00:00, 1958.58it/s]


In [11]:
23485 / 3

7828.333333333333

In [4]:
def split_validation_set(source_dir, target_dir, val_count=7829, seed=42, move=False):

    os.makedirs(target_dir, exist_ok=True)
    
    all_images = [f for f in os.listdir(source_dir) if f.lower().endswith('.jpg')]
    
    if val_count > len(all_images):
        raise ValueError(f"Se solicitaron {val_count} imágenes, pero solo hay {len(all_images)} disponibles.")
    
    random.seed(seed)
    selected_images = random.sample(all_images, val_count)
    
    for filename in selected_images:
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        if move:
            shutil.move(src_path, dst_path)
        else:
            shutil.copy2(src_path, dst_path)

    print(f"{val_count} imágenes {'movidas' if move else 'copiadas'} a '{target_dir}'.")

In [13]:
split_validation_set(
    source_dir='../data/raw/subset/train2014/train2014',
    target_dir='../data/raw/subset/val2014',
    val_count=7829,
    move=True  
)

7829 imágenes movidas a '../data/raw/subset/val2014'.


In [5]:
split_validation_set(
    source_dir='../data/raw/subset/val2014',
    target_dir='../data/raw/subset/test2014',
    val_count=700,
    move=True  
)

700 imágenes movidas a '../data/raw/subset/test2014'.
